In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pickle 
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
from skimage import draw
import pandas as pd
from torchsummary import summary
from scipy import ndimage 
import sys

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/xs/rs12w90s1_16jbgdwmt4z_7r0000gn/T/ipykernel_33708/2786327518.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers, drop_rate=0.4, b_con = True):
        super(DenseBlock, self).__init__()
        if b_con:
            self.layers = nn.ModuleList([self._make_layer(in_channels + i * growth_rate, growth_rate) for i in range(num_layers)])
        else:
            self.layers = nn.ModuleList([self._make_layer(in_channels, growth_rate) if i==0 else self._make_layer(growth_rate, growth_rate) for i in range(num_layers)])
        self.dropout = nn.Dropout2d(drop_rate)
        self.b_con = b_con

    def _make_layer(self, in_channels, growth_rate):
        return nn.Sequential(
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1),
            nn.BatchNorm2d(growth_rate),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        for layer in self.layers:
            if self.b_con:
                temp = x
            x = layer(x)
            x = self.dropout(x)
            if self.b_con:
                x = torch.cat([x, temp], 1)
        return x

class TransitionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        return x

class DenseNet_2block(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16), num_init_features=3, num_classes=3, drop_rate=0.4, block_connection =[True,True]):
        super(DenseNet_2block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(num_init_features),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        in_channels = num_init_features
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(in_channels, growth_rate, num_layers, drop_rate, b_con = block_connection[i])
            self.features.add_module(f'denseblock{i + 1}', block)
            if block_connection[i]:
                in_channels = in_channels + num_layers * growth_rate
            else:
                in_channels =  growth_rate 
            # in_channels = in_channels + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = TransitionBlock(in_channels, in_channels // 2)
                self.features.add_module(f'transition{i + 1}', trans)
                in_channels = in_channels // 2

        self.features.add_module('norm5', nn.BatchNorm2d(in_channels))
        self.features.add_module('relu5', nn.ReLU(inplace=True))
        self.features.add_module('avgpool5', nn.AdaptiveAvgPool2d((1, 1)))

        self.classifier = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        features = self.features(x)
        out = torch.flatten(features, 1)
        out = self.classifier(out)
        return out
    
class DenseNet_3block(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24), num_init_features=3, num_classes=3, drop_rate=0.4, block_connection =[True,True,True]):
        super(DenseNet_3block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(num_init_features),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        in_channels = num_init_features
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(in_channels, growth_rate, num_layers, drop_rate, b_con = block_connection[i])
            self.features.add_module(f'denseblock{i + 1}', block)
            if block_connection[i]:
                in_channels = in_channels + num_layers * growth_rate
            else:
                in_channels = growth_rate 
            # in_channels = in_channels + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = TransitionBlock(in_channels, in_channels // 2)
                self.features.add_module(f'transition{i + 1}', trans)
                in_channels = in_channels // 2

        self.features.add_module('norm5', nn.BatchNorm2d(in_channels))
        self.features.add_module('relu5', nn.ReLU(inplace=True))
        self.features.add_module('avgpool5', nn.AdaptiveAvgPool2d((1, 1)))

        self.classifier = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        features = self.features(x)
        out = torch.flatten(features, 1)
        out = self.classifier(out)
        return out

In [5]:
blcs = [True, True] # block connection
# b_config = (6, 8, 8, 12) # block configuration
b_config = (16,14)
gr = 64 # growth rate
dr = 0.4 # drop rate
num_cls = 2 # number of classes
epochs = 300 # number of epochs
b_size = 512 # batch size
lr = 1e-3 # learning rate
l2_norm = 1e-4 # l2 norm
gamma = 0.99
step_size = 1
# model = DenseNet(growth_rate=gr, block_config=b_config, drop_rate=dr, num_init_features=3, num_classes=num_cls, block_connection=blcs)
model = DenseNet_2block(growth_rate=gr, block_config=b_config, drop_rate=dr, num_init_features=3, num_classes=num_cls, block_connection=blcs)
# model = DenseNet_3block(growth_rate=gr, block_config=b_config, drop_rate=dr, num_init_features=3, num_classes=num_cls, block_connection=blcs)
ds = summary(model, (3, 80, 80))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 1409, 1, 1]          --
|    └─Conv2d: 2-1                       [-1, 3, 40, 40]           444
|    └─BatchNorm2d: 2-2                  [-1, 3, 40, 40]           6
|    └─ReLU: 2-3                         [-1, 3, 40, 40]           --
|    └─MaxPool2d: 2-4                    [-1, 3, 20, 20]           --
|    └─DenseBlock: 2-5                   [-1, 1027, 20, 20]        --
|    |    └─Dropout2d: 3-1               [-1, 64, 20, 20]          --
|    |    └─Dropout2d: 3-2               [-1, 64, 20, 20]          --
|    |    └─Dropout2d: 3-3               [-1, 64, 20, 20]          --
|    |    └─Dropout2d: 3-4               [-1, 64, 20, 20]          --
|    |    └─Dropout2d: 3-5               [-1, 64, 20, 20]          --
|    |    └─Dropout2d: 3-6               [-1, 64, 20, 20]          --
|    |    └─Dropout2d: 3-7               [-1, 64, 20, 20]          --
|    |    └─Dro